# Correlations

This file contains all of the correlations that we want to calculate. This means that we need at least 2 columns to create a result.

Most of these correlations have to do with coop salary, or term average. We use these as metrics of student success because they're numeric, making them easier to process. Other properties are very subjective. Salary and grades are not the most indicative of how successful a student is, but with the existing data, it's the best indication we have.

In [1]:
from IPython.display import display

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import json
from collections import defaultdict

from Bucket import Bucket
from Distribution import Distribution
from GradeSalaryHistory import GradeSalaryHistory


pd.set_option('display.max_columns', None)
plt.style.use('ggplot')

# Show matplotlib plots in this notebook
%matplotlib inline
# Setting plot parameters
from pylab import rcParams
params = {
    'figure.figsize': (8, 8),
    'legend.fontsize': 15
}
rcParams.update(params)

In [2]:
def isnan(a):
    return a != a

In [3]:
def isempty(a):
    return isnan(a) or not a

In [4]:
def isfloat(value):
  try:
    float(value)
    return not isnan(value)
  except ValueError:
    return False

In [5]:
# Where to write the buckets
BUCKET_DIR = '../private/buckets/'
GRADE_SALARY_FILENAME = '../private/grade-vs-salary.json'
df = pd.read_csv('../private/results-04-10.csv')  # TODO: Write the response file

COOP = ['1', '2', '3', '4', '5', '6']
TERM = ['1a', '1b', '2a', '2b', '3a', '3b', '4a']
MULTI_VAL_COL = ['ethnicity', 'fav_lang', 'preferred_tech_discipline', 'text_editor']

SALARY_COL = ['coop_salary_' + i + '.csv' for i in COOP]
GRADE_COL = ['term_avg_' + i + '.csv' for i in TERM]

In [6]:
def write_buckets(df, col_name):
    """Creates a bucket for each value and then writes the values into a file."""
    buckets = correlate_columns(df, col_name)
    with open(BUCKET_DIR + col_name + '_buckets.json', 'w') as f:
        result = {}
        for i in buckets:
            result[i] = buckets[i].summary()
        f.write(json.dumps(result, indent=True))

In [12]:
def correlate_columns(df, col_name):

    """Generates a bucket for unique column value."""
    # Get unique values (some rows have multiple values)
    col_values = np.array([])
    for i in df[col_name]:
        if isfloat(i):
            # Floor any data
            i = math.floor(float(i))
        if isnan(i):
            # Don't want to include any NaN
            print('Skipping ', i)
            continue
            
        # Append separate values if it's a comma separated value
        val = str(i)
        if ',' in val and col_name in MULTI_VAL_COL:
            col_values = np.append(col_values, list(map(str.strip, val.split(','))))
        else:
            col_values = np.append(col_values, val)
            
    col_values = np.unique(col_values)
    # Create a bucket for each column value
    buckets = {}
    for col_val in col_values:
        # Initialize distributions
        salaries = {}
        grades = {}
        for i in TERM:
            grades[i] = np.array([])
        for i in COOP:
            salaries[i] = np.array([])
        
        # Iterate through all rows
        for i in range(0, df.shape[0]):
            val = df[col_name][i]
            if isfloat(val):
                val = math.floor(float(val))
            
            # Filter any NaN or non matching values
            if isnan(val):
                continue
            if col_name in MULTI_VAL_COL:
                if str(col_val) not in map(str.strip, str(val).split(',')):
                    continue
            else:
                if str(col_val) != str(val):
                    continue
                
            # Add grades
            for t in TERM:
                avg = df['term_avg_' + t][i]
                if avg == 'exchange':
                    continue
                if isnan(avg):
                    continue
                grades[t] = np.append(grades[t], avg)
            
            # Add salary
            for c in COOP:
                salary = df['coop_salary_' + c][i]
                if type(salary) == float and math.isnan(salary):
                    continue
                if ',' in salary:
                    salary = salary.replace(',', '')
                salaries[c] = np.append(salaries[c], salary)

        # Create the bucket
        buckets[col_val] = Bucket(col_name, col_val, 
                   [Distribution(grades[i].astype(float)) for i in TERM],
                   [Distribution(salaries[i].astype(float)) for i in COOP])
    
    return buckets

In [14]:
# From https://github.com/se2018/class-profile/tree/master/analyses
to_correlate = [
    'gender',
    'ethnicity',
    'family_income',
    'work_os',
    'phone',
    'soft_eng_rating',
    'se_friendships',
    'is_international',
    'parents_edu',
    'parents_technical',
    'admission_avg',
    'code_start_age',
    'fav_lang',
    'num_hackathons',
    'side_proj',
    'exercise',
    'cooking',
    'sleep_time',
    'preferred_tech_discipline',
    'text_editor'
]

for i in to_correlate:
    write_buckets(df, i)

('Skipping ', nan)
('Skipping ', nan)
('Skipping ', nan)
('Skipping ', nan)
('Skipping ', nan)
('Skipping ', nan)


Next section is to get the correlation between grades and coop jobs.

In [29]:
def create_history(df):
    """Creates an entry for each coop term about the grades and salaries leading up to it."""
    result = {}
    for i, term in enumerate(COOP):
        result[term] = defaultdict(list)
        
        for row in range(0, df.shape[0]):
            # Skip any entries that are missing data on the coop
            if isempty(df['coop_name_' + term][row]) or isempty(df['coop_salary_' + term][row]):
                print 'Empty term for row ', row, ', term ', term, 'skipping entry.'
                continue
                
            # Process a coop term
            term_avgs = np.array([])
            salaries = np.array([])
            
            # Get previous grades
            for study in range(0, i+1):
                val = df['term_avg_' + str(TERM[study])][row]
                if isnan(val) or val == 'exchange':
                    term_avgs = np.append(term_avgs, 0.0)
                else:
                    term_avgs = np.append(term_avgs, math.floor(float(val)))
            
            # Get previous salaries
            for coop in range(0, i):
                val = str(df['coop_salary_' + COOP[coop]][row])
                val = val.replace(',', '')
                if isnan(float(val)):
                    salaries = np.append(salaries, 0)
                else:
                    salaries = np.append(salaries, float((int(float(val)) / 500 * 500)))
            
            location = df['coop_loc_' + term][row]
            if isempty(location):
                location = ''
            salary = df['coop_salary_' + term][row]
            # Estimate to the nearest 500*i
            salary = float(int(float(str(salary).replace(',', ''))) / 500 * 500)
            
            result[term][str(salary)].append(GradeSalaryHistory(term_avgs, salaries, location, salary).summary())
    
    return result
            

In [30]:
with open(GRADE_SALARY_FILENAME, 'w') as f:
    f.write(json.dumps(create_history(df), indent=True))

Empty term for row  5 , term  1 skipping entry.
Empty term for row  30 , term  1 skipping entry.
Empty term for row  45 , term  1 skipping entry.
Empty term for row  55 , term  1 skipping entry.
Empty term for row  60 , term  1 skipping entry.
Empty term for row  67 , term  1 skipping entry.
Empty term for row  93 , term  1 skipping entry.
Empty term for row  7 , term  2 skipping entry.
Empty term for row  45 , term  2 skipping entry.
Empty term for row  64 , term  2 skipping entry.
Empty term for row  82 , term  3 skipping entry.
Empty term for row  91 , term  3 skipping entry.
Empty term for row  100 , term  3 skipping entry.
Empty term for row  8 , term  4 skipping entry.
Empty term for row  73 , term  4 skipping entry.
Empty term for row  107 , term  4 skipping entry.
Empty term for row  112 , term  4 skipping entry.
Empty term for row  26 , term  5 skipping entry.
Empty term for row  77 , term  5 skipping entry.
Empty term for row  86 , term  5 skipping entry.
Empty term for row  